In [12]:
import os, sys, inspect
# sys.path.append('../..')

import pandas as pd
from pathlib import Path
import glob as glob
import os
import numpy as np
import re
import json
from unidecode import unidecode
from datetime import datetime, timedelta, date
from connection import SCORE_DB_CONN, ANALYTICS_DB_CONN
from avay_bq import AvayBQServiceAcc
from da_utils.client.bigquery_client import BigQueryClient
from da_utils.client.google_sheets_client import GoogleSheetsClient
from da_utils.repository.google_sheets.google_sheets_repository import GoogleSheetsRepository
from da_utils.repository.bigquery.bigquery_repository import BigqueryRepository



pd.set_option("display.max_row", 100)

In [13]:
adc_path = '/home/linhnguyen/application_default_credentials.json'
avay_bq_acc = AvayBQServiceAcc()
bq_client = BigQueryClient(billing_project_id='prj-ts-p-analytic-8057', adc_file_path_str=adc_path)
bq_repo = BigqueryRepository(bq_client)

/home/linhnguyen/.venv/lib/python3.9/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [15]:

lead_phone_infos = pd.read_sql(
    """
    with pre as (
select id as lead_id, 
case when (other->>'sender' in ('sms','click') or other->>'source' like '%%sms%%') then 'SMS' else 'AVAY' end as channel,
bound_code as offer_code,
to_char(sent_at at time zone 'utc+7', 'YYYY-MM') as month,
score_range,
left(score_range,3)::numeric as min_score,
right(score_range,3)::numeric as max_score,
telco_code
from phone_infos
where 1=1
and bank_code in ('lotte')
and sent_at at time zone 'utc+7' >= '2022-01-01'
and sent_at at time zone 'utc+7' < '2023-03-01'
and pre_scoring_data->>'is_qualified'='true'
and was_sent = true)

select *,
case when min_score <= 631 and max_score <= 631 then '[600-630]' else '[631+)' end as score_range2
from pre
""",
    SCORE_DB_CONN,
)

In [19]:
disbursed = pd.read_sql(
    """
select lead_id, 
    case when bank_code = 'lotte' then batch_date else disbursed_date end as loan_date, 
    (select max(disbursed_date) from leadgen.disbursals) report_date,
    -- '2023-01-31'::date AS report_date,
    loan_amount,
    commission
from leadgen.disbursals
where true and bank_code = 'lotte'
""",
    ANALYTICS_DB_CONN,
)

In [20]:
mapping = lead_phone_infos.merge(disbursed, how = 'left')
mapping['disbursed'] = np.where(mapping['loan_date'].notnull(),1,0)
mapping


,lead_id,channel,offer_code,month,score_range,min_score,max_score,telco_code,score_range2,loan_date,report_date,loan_amount,commission,disbursed
0,13330794,SMS,LOTTE02,2022-11,660-669,660.0,669.0,viettel,[631+),2022-11-29,2023-01-31,NaN,280000.0,1
1,13501747,AVAY,LOTTE02,2023-01,610-619,610.0,619.0,viettel,[600-630],2023-01-03,2023-01-31,NaN,168000.0,1
2,13502062,AVAY,LOTTE02,2023-01,760-769,760.0,769.0,viettel,[631+),2023-01-03,2023-01-31,NaN,168000.0,1
3,13501744,AVAY,LOTTE02,2023-01,640-649,640.0,649.0,viettel,[631+),2023-01-03,2023-01-31,NaN,168000.0,1
4,13501863,AVAY,LOTTE02,2023-01,750-759,750.0,759.0,viettel,[631+),2023-01-03,2023-01-31,NaN,168000.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88713,13729582,AVAY,LOTTE02,2023-02,720-729,720.0,729.0,viettel,[631+),NaN,NaN,NaN,NaN,0
88714,13729333,AVAY,LOTTE02,2023-02,610-619,610.0,619.0,viettel,[600-630],NaN,NaN,NaN,NaN,0
88715,13729539,AVAY,LOTTE02,2023-02,680-689,680.0,689.0,viettel,[631+),NaN,NaN,NaN,NaN,0
88716,13729373,AVAY,LOTTE02,2023-02,600-609,600.0,609.0,viettel,[600-630],NaN,NaN,NaN,NaN,0


In [21]:
lfvnlead = mapping.groupby(["channel", "offer_code", "score_range2","month","telco_code"]).agg({"lead_id":"count", "disbursed": "sum"}).reset_index()

lfvnlead

,channel,offer_code,score_range2,month,telco_code,lead_id,disbursed
0,AVAY,LOTTE00,[631+),2023-02,mobifone,86,0
1,AVAY,LOTTE00,[631+),2023-02,vietnamobile,6,0
2,AVAY,LOTTE00,[631+),2023-02,vinaphone,128,0
3,AVAY,LOTTE02,[600-630],2022-08,viettel,99,30
4,AVAY,LOTTE02,[600-630],2022-09,viettel,285,110
5,AVAY,LOTTE02,[600-630],2022-10,viettel,431,118
6,AVAY,LOTTE02,[600-630],2022-11,viettel,1144,376
7,AVAY,LOTTE02,[600-630],2022-12,viettel,2596,945
8,AVAY,LOTTE02,[600-630],2023-01,viettel,1692,632
9,AVAY,LOTTE02,[600-630],2023-02,viettel,1172,0


In [22]:
client = GoogleSheetsClient()
repo = GoogleSheetsRepository(client)
workbook = repo.open_spreadsheet_by_key("1BCn3B2uk6cZeUW38fG6Uc9A8b3IehXmIXc8Bo0OJwns")
repo.write_df_to_sheet(workbook.worksheet("data"),lfvnlead)